<a href="https://colab.research.google.com/github/deonGuo/shroom_oracle/blob/main/shroom_oracle_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *
from ipywidgets import widgets, VBox, Label, Output
from IPython.display import display

# Load model
learn = load_learner('oracle.pkl', cpu=True)

# Widgets
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Pray to Shroom Oracle')
out_pl = Output()
lbl_prediction = Label()
debug_output = Output()

# Event handler
def on_click_classify(change):
    with debug_output:
        debug_output.clear_output()
        print("Button clicked")

        try:
            upload_val = btn_upload.value
            if isinstance(upload_val, dict):
                uploaded_file = list(upload_val.values())[0]
            elif isinstance(upload_val, tuple):
                uploaded_file = upload_val[0]
            else:
                raise Exception("Unsupported upload type")

            content = uploaded_file['content']
            print("Uploaded file keys:", uploaded_file.keys())
            print(f"Received file: {uploaded_file['name']}, size: {len(content)} bytes")

            # Convert memoryview to bytes
            img = PILImage.create(bytes(content))
            print("Image loaded")

            pred, pred_idx, probs = learn.predict(img)
            print(f"Prediction: {pred}, Probability: {probs[pred_idx]:.4f}")

            out_pl.clear_output()
            with out_pl:
                display(img.to_thumb(128, 128))
            lbl_prediction.value = f"Prediction: {pred}; Probability: {probs[pred_idx]:.04f}"

        except Exception as e:
            print("Error:", e)
            lbl_prediction.value = f"Error: {e}"

# Bind event
btn_run.on_click(on_click_classify)

# Layout
ui = VBox([
    Label("🍄 Behold, O The Great Shroom Oracle"),
    btn_upload,
    btn_run,
    out_pl,
    lbl_prediction,
    Label("🔧 Debug Console"),
    debug_output
])
display(ui)
